In [1]:
import OpenDartReader
import pandas as pd 
import os
import glob
import numpy as np

In [ ]:
# df = pd.read_csv('esg_financial_data_9_columns.csv', 
#                  dtype={'corp_code': str, 'stock_code': str})
# c_code = df['stock_code'].to_list()
# stock_code = [str(code) for code in c_code]

# def add_zero(c):
#     if len(c) < 6:
#         c = (6-len(c)) * '0' + c
#     return c

# st_code = list(map(add_zero, stock_code))
# df['stock_code'] = st_code

# c_code = df['corp_code'].to_list()
# corp_code = [str(code) for code in c_code]

# def add_zero(c):
#     if len(c) < 8:
#         c = (8-len(c)) * '0' + c
#     return c

# st_code = list(map(add_zero, corp_code))
# df['corp_code'] = st_code

In [ ]:
# api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)

df = pd.read_csv('ESG_data.csv', index_col=0, dtype={'corp_code': str,
                                                       'stock_code': str})
print(df.shape)
df.head()

In [ ]:
entire_codes = []

for f in glob.glob('./2020/fin_2020/*.csv'):
    code = f.split('\\')[1][:6]
    entire_codes.append(code)

code_missing = [code for code in list(df['stock_code'].unique()) if code not in entire_codes]

print('저장된 회사 수: ', len(entire_codes)) # 670개 데이터 존재
print('누락 회사 수: ', len(code_missing)) # 124개는 다시 호출할 필요 있음

### 무형자산 데이터 import

In [ ]:
# csv 파일 합치기

df_c = df.copy()

df_c['intangible'] = np.zeros(df_c.shape[0])

code_missing = [code for code in list(df['stock_code'].unique()) if code in entire_codes]

missing_codes_intang = []
for code in code_missing:
    try:
        docu = pd.read_csv(f'./2022/fin_2022/{code}_2022.csv', index_col=0)
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code]

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
    except:
        missing_codes_intang.append(code)

df_c['intangible'] = pd.to_numeric(df_c['intangible'])

In [ ]:
# 누락된 데이터는 OpenDartreader 다시 이용
# 사업 보고서 > 연결재무제표
re_codes = df_c[df_c['intangible'] == 0]['stock_code'].unique()

re = []
for code in re_codes:
    try: 
        lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
        report_num = lst[(lst['report_nm'].str.contains('사업보고서')) & (lst['report_nm'].str.contains('2022'))].iloc[0, 5]
        url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
        html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

        docu = pd.read_html(html)[1]

        docu = docu.rename(columns={'Unnamed: 0': 'tag'})
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code]

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
        print(f'{code}: good')
    except:
        re.append(code)
print(len(re))

In [ ]:
# 사업보고서 > 영업보고서

re_re = []
for code in re:
    try:
        lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
        report_num = lst[(lst['report_nm'].str.contains('사업보고서')) & (lst['report_nm'].str.contains('2022'))].iloc[0, 5]
        url = dart.sub_docs(report_num).iloc[1,1] #사업보고서 번호를 통해 url 요청
        # html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

        docu = pd.read_html(html)[1]

        docu = docu.rename(columns={'Unnamed: 0': 'tag'})
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code]

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
        print(f'{code}: good')
    except:
        re_re.append(code)

In [ ]:
df_c[df_c['intangible'] == 0]

In [ ]:
# 무형자산 import 완료 # save point 
df_c.to_csv('./ESG_data_intangible.csv', encoding='utf8', index = False)

### 현금배당성향 import

In [165]:
df = pd.read_csv('ESG_data_intangible.csv', index_col=0, dtype={'corp_code': str,
                                                       'stock_code': str})
entire_codes = []

for f in glob.glob('./2022/cash_2022/*.csv'):
    code = f.split('\\')[1][:8]
    entire_codes.append(code)

code_missing = [code for code in list(df['corp_code'].unique()) if code not in entire_codes]

print('저장된 회사 수: ', len(entire_codes)) # 690개 데이터 존재
print('누락 회사 수: ', len(code_missing)) # 3 개는 다시 호출할 필요 있음

저장된 회사 수:  844
누락 회사 수:  3


In [166]:
df_s = df.copy()
df_s = df_s.reset_index()

df_s['ratio'] = np.nan

# 1단계 csv 파일 합치기
re_codes= []
for code in df_s['corp_code'].unique():
    try:
        docu = pd.read_csv(f'./2022/cash_2022/{code}_2022.csv', index_col=0)
        cash = docu[docu['se']=='(연결)현금배당성향(%)'].iloc[0, 2:].values
        
        data = df_s[df_s['corp_code'] == code]

        for i, (k,v) in enumerate(data.ratio.items()):
            df_s.loc[k, 'ratio'] = cash[i]
    except:
        re_codes.append(code)
print(sum(df_s['ratio'].isna()))

9


In [167]:
# 2단계: 함수 불러오기
# api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)

for code in re_codes:
    print(code)
    docu = dart.report(code, '배당', 2022)
    
    cash = docu[docu['se'].str.contains('현금배당성향')].iloc[0, -3:].values
        
    data = df_s[df_s['corp_code'] == code]

    for i, (k,v) in enumerate(data.ratio.items()):
        df_s.loc[k, 'ratio'] = cash[i]
        
print(sum(df_s['ratio'].isna())) # ratio 반영 완료 # nan 값 하나가 재무재표에 있음

00115287
00545929
00530185
1


In [168]:
print(sum(df_s['ratio']=='-')) # 862

df_s['ratio'] = df_s['ratio'].replace('-', np.nan) # '-'를 nan값으로 변환
df_s['ratio'] = df_s['ratio'].replace('#######', np.nan) # '#######'를 nan값으로 변환
df_s['ratio'] = df_s['ratio'].astype(float) # error

# ratio 반영 완료 # nan 값 다수 존재 -> 대체로 평균을 사용할 가능성

862


In [169]:
# pre_ratio
df_s['pre_ratio'] = np.nan

for code in df_s['corp_code'].unique():
    ratio_index = df_s[df_s['corp_code'] == code]['ratio'].index
    v = df_s[df_s['corp_code'] == code]['ratio'].values
    
    docu = dart.report(code, '배당', 2020)    
    v_2019 = docu[docu['se']=='(연결)현금배당성향(%)'].iloc[0, -2] 
    
    df_s.loc[ratio_index[0], 'pre_ratio'] = v[1]
    df_s.loc[ratio_index[1], 'pre_ratio'] = v[2]
    df_s.loc[ratio_index[2], 'pre_ratio'] = v_2019

print(sum(df_s['pre_ratio'] =='-')) # 279

279


In [170]:
df_s['pre_ratio'] = df_s['pre_ratio'].replace('-', np.nan)
df_s['pre_ratio'] = df_s['pre_ratio'].replace('#######', np.nan)
df_s['pre_ratio'] = df_s['pre_ratio'].astype(float)

In [171]:
# Final checking # 298000 # 효성화학 문제
lst = dart.list('298000', start='2020', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
report_num = lst[(lst['report_nm'].str.contains('사업보고서')) & (lst['report_nm'].str.contains('2022'))].iloc[0, 5]

url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

docu = pd.read_html(html)[1]

docu = docu.rename(columns={'Unnamed: 0': 'tag'})
intang = docu[docu['tag'] == '무형자산'].iloc[0, 1:].values

data = df_s[df_s['stock_code']=='298000']
d_index = data['intangible'].index

df_s.loc[d_index[0], 'intangible'] = intang[0]
df_s.loc[d_index[1], 'intangible'] = intang[1]
df_s.loc[d_index[2], 'intangible'] = intang[2]

df_s[df_s['stock_code']=='298000']

,회사명,corp_code,stock_code,연도,ESG,재무제표,부채총계,영업이익,법인세차감전 순이익,intangible,ratio,pre_ratio
143,효성화학,01316236,298000,2022,A,"3,131,118,899,043","3,016,501,814,422","-336,727,239,866","-448,250,891,421",32207264388,NaN,NaN
933,효성화학,01316236,298000,2021,A,"3,044,049,458,130","2,554,748,674,581","136,574,029,923","102,372,123,415",33970502392.0,NaN,NaN
1723,효성화학,01316236,298000,2020,A,"2,417,365,034,469","2,015,011,905,019","60,927,548,911","3,421,353,521",23126250265,NaN,15.6


In [172]:
df_s.info() 
# intangible: 무형자산를 nan값으로 공시한 회사 있음 2019
# ratio, pre_ratio는 '-' 값 많음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2370 entries, 0 to 2369
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   회사명         2370 non-null   object 
 1   corp_code   2370 non-null   object 
 2   stock_code  2370 non-null   object 
 3   연도          2370 non-null   int64  
 4   ESG         2370 non-null   object 
 5   재무제표        2370 non-null   object 
 6   부채총계        2370 non-null   object 
 7   영업이익        2370 non-null   object 
 8   법인세차감전 순이익  2370 non-null   object 
 9   intangible  2369 non-null   object 
 10  ratio       1502 non-null   float64
 11  pre_ratio   1506 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 222.3+ KB


In [173]:
# intangible, ratio, pre_ratio 완료
df_s.to_csv('esg_data_intang_ratio.csv', encoding='utf8')